In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.externals import joblib


def get_data_dir():
    return '../data/'


def merged_file_path():
    return get_data_dir()+'Merged_Data_Excel_Test.xlsx'


def train_file_path():
    return get_data_dir()+'train_data.pkl'


def scaler_file_path():
    return get_data_dir()+'scaler.pkl'


def model_file_path():
    return get_data_dir()+'/models/'+'/model-v4-epoch-53-val_loss-0.00.h5'


df = pd.read_excel(merged_file_path(),index_col=0,parse_dates=True )
df.ffill(inplace=True)
print("Rows in data : "+str(len(df)))

scaler = joblib.load(scaler_file_path())
#w_data = df.as_matrix()

scaled = scaler.transform(df)

timesteps=60
rowcount = scaled.shape[0]
close_price_col_idx=0

X_list=[]
Y_list=[]


for i in range(timesteps,rowcount,1):
    target_row=scaled[i]
    y_val = target_row[close_price_col_idx]
    start_row=i-timesteps
    end_row=i

    Y_list.append(y_val)
    x=scaled[start_row:end_row]
    x_first_row=x[0]
    x_last_row = x[timesteps-1]
    X_list.append(x)


    
X=np.asarray(X_list)
Y_hat=np.asarray(Y_list)

num_features=X.shape[2]


print(X.shape)
print(Y_hat.shape)

print("Data prepared")

model = load_model(model_file_path())


d:\work\lstm_stock_price\venv_cpu\lib\site-packages\sklearn\externals\joblib\__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


Rows in data : 8917


W0625 22:53:57.160215 16852 deprecation.py:506] From d:\work\lstm_stock_price\venv_cpu\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0625 22:53:57.161213 16852 deprecation.py:506] From d:\work\lstm_stock_price\venv_cpu\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0625 22:53:57.162209 16852 deprecation.py:506] From d:\work\lstm_stock_price\venv_cpu\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_

(8857, 60, 14)
(8857,)
Data prepared


W0625 22:53:57.783549 16852 deprecation.py:323] From d:\work\lstm_stock_price\venv_cpu\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [2]:
close_prices=scaled[timesteps:,close_price_col_idx]

In [3]:
timestamps=df.index.values[timesteps:]

In [4]:
cp_df=pd.DataFrame(data=close_prices,columns=['close_price'])


In [5]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import cufflinks as cf

init_notebook_mode(connected=True)
cf.go_offline()

In [6]:
cp_df.iplot()

In [7]:
y_all = model.predict(X, batch_size=32)

In [8]:
cp_df['forecast']=np.asarray(y_all)

In [9]:
cp_df['date'] = timestamps
cp_df = cp_df.set_index('date')

In [10]:
cp_df.iplot()